# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
    git status

On branch pr/ashdar/752
Your branch is ahead of 'ashdar/fix_PingComputer' by 66 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1
$null= Reset-DbcConfig

Excellent, that worked. Now to run some checks against a single instance  

I don't care if the test pass or fail, I am only interested in any 

> Describe block has Error

or

> Error in Script

type failures

Multiple Instances at parameter

In [3]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Invoke-DbcCheck -ComputerName $Sqlinstances -Check PingComputer

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1' with Tags PingComputer

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1

  Describing Ping Computer

    Context Testing Ping to SQL2005Ser2003
      [+] Should have pinged 3 times for SQL2005Ser2003 58ms
      [+] Average response time (ms) should Be less than 10 (ms) for SQL2005Ser2003 22ms

  Describing Ping Computer

    Context Testing Ping to SQL2016N2
      [+] Should have pinged 3 times for SQL2016N2 8ms
      [+] Average response time (ms) should Be less than 10 (ms) for SQL2016N2 9ms

  Describing Ping Computer

    Context Testing Ping to SQL2012Ser08AG1
      [+] Should have pinged 3 times for SQL2012Ser08AG1 7ms
      [+] Average response time (ms) should Be less than 10 (ms) for SQL2012Ser08AG

Set the config and run again

In [8]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Set-DbcConfig -Name skip.security.sadisabled -Value $false
Invoke-DbcCheck -SqlInstance $Sqlinstances -Check SaDisabled


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1' with Tags SaDisabled

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2005Ser2003
      [+] sa login is disabled on SQL2005Ser2003 50ms

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2016N2
      [+] sa login is disabled on SQL2016N2 2ms

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2012Ser08AG1
      [+] sa login is disabled on SQL2012Ser08AG1 2ms

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2016N3
      [+] sa login is disabled on SQL2016N3 3ms

  Describing SA Login Disable

In [11]:
Get-DbaServerRole -SQLInstance sql2019n5 -ServerRole "sysadmin"


ComputerName : SQL2019N5
InstanceName : MSSQLSERVER
SqlInstance  : SQL2019N5
Role         : sysadmin
Login        : {sa, THEBEARD\Rob, NT SERVICE\SQLWriter, NT SERVICE\Winmgmt…}
Owner        : sa
IsFixedRole  : True
DateCreated  : 13/04/2009 12:59:06
DateModified : 13/04/2009 12:59:06




In [9]:
@(Get-DbaLogin -SQLInstance sql2019n5 -Type SQL | Where-Object { $_.PasswordPolicyEnforced -eq $false -and $_.IsDisabled -eq $false}).Count

1


In [10]:
@(Get-DbaLogin -SqlInstance sql2019n5 -Login sa).Count

1
